In [1]:
import pandas as pd
import numpy as np
# from sklearn.preprocessing import StandardScaler
from prophet import Prophet


c:\Users\Usuario\.conda\envs\pylab311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Importing plotly failed. Interactive plots will not work.


In [2]:
ESTOY_EN_KAGGLE = True

In [3]:
if ESTOY_EN_KAGGLE:
    df = pd.read_csv("./datasets/periodo_x_producto_con_target_transformado_201912.csv", sep=',', encoding='utf-8')
else:
    df = pd.read_csv("./datasets/periodo_x_producto_con_target_transformado.csv", sep=',', encoding='utf-8')
df

,product_id,periodo,nacimiento_producto,muerte_producto,mes_n,total_meses,producto_nuevo,ciclo_de_vida_inicial,sku_size,stock_final,...,brand_nan,tn_zscore,stock_final_zscore,cust_request_qty_zscore,cust_request_tn_zscore,target,tn_log,stock_final_log,cust_request_qty_log,cust_request_tn_log
0,20001,201701,201701,201912,1,36,0,0,3000.0,NaN,...,0.0,-1.576908,NaN,0.461891,-1.704215,-0.465448,6.841372,NaN,6.173786,6.844525
1,20001,201702,201701,201912,2,36,0,0,3000.0,NaN,...,0.0,-2.042103,NaN,-0.046911,-2.039835,-1.325726,6.683381,NaN,6.070738,6.727099
2,20001,201703,201701,201912,3,36,0,0,3000.0,NaN,...,0.0,-0.323111,NaN,0.786658,-0.435961,0.267471,7.173466,NaN,6.234411,7.194247
3,20001,201704,201701,201912,4,36,0,0,3000.0,NaN,...,0.0,-1.117043,NaN,-1.703222,-1.074260,0.333316,6.976312,NaN,5.634790,7.033457
4,20001,201705,201701,201912,5,36,0,0,3000.0,NaN,...,0.0,0.353285,NaN,2.865166,0.273781,-1.470536,7.315352,NaN,6.553933,7.347100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31357,21281,201704,201702,201708,3,7,1,1,NaN,NaN,...,1.0,-0.670410,NaN,NaN,NaN,1.850294,0.000000,NaN,NaN,NaN
31358,21281,201705,201702,201708,4,7,1,0,NaN,NaN,...,1.0,-0.670410,NaN,NaN,NaN,-0.707610,0.000000,NaN,NaN,NaN
31359,21281,201706,201702,201708,5,7,1,0,NaN,NaN,...,1.0,2.237248,NaN,1.414214,1.397345,0.272537,0.087406,NaN,2.197225,0.100198
31360,21281,201707,201702,201708,6,7,1,0,NaN,NaN,...,1.0,-0.670410,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN


In [5]:
#### Revisar si hay infinitos en la columna target
print("Valores infinitos en 'tn_zscore':", np.isinf(df['tn_zscore']).sum())
# Revisar también NaN
print("Valores NaN en 'tn_zscore':", df['tn_zscore'].isna().sum())
df['tn_zscore'] = df['tn_zscore'].replace([np.inf, -np.inf], np.nan)
# df['tn_zscore'] = df['tn_zscore'].fillna(df['tn_zscore'].median())  # Usar la mediana
df['tn_zscore'] = df['tn_zscore'].fillna(0)  # Usar cero #### EXPERIMENTAR CON ESTO
df

Valores infinitos en 'tn_zscore': 0
Valores NaN en 'tn_zscore': 0


,product_id,periodo,nacimiento_producto,muerte_producto,mes_n,total_meses,producto_nuevo,ciclo_de_vida_inicial,sku_size,stock_final,...,brand_nan,tn_zscore,stock_final_zscore,cust_request_qty_zscore,cust_request_tn_zscore,target,tn_log,stock_final_log,cust_request_qty_log,cust_request_tn_log
0,20001,201701,201701,201912,1,36,0,0,3000.0,NaN,...,0.0,-1.576908,NaN,0.461891,-1.704215,-0.465448,6.841372,NaN,6.173786,6.844525
1,20001,201702,201701,201912,2,36,0,0,3000.0,NaN,...,0.0,-2.042103,NaN,-0.046911,-2.039835,-1.325726,6.683381,NaN,6.070738,6.727099
2,20001,201703,201701,201912,3,36,0,0,3000.0,NaN,...,0.0,-0.323111,NaN,0.786658,-0.435961,0.267471,7.173466,NaN,6.234411,7.194247
3,20001,201704,201701,201912,4,36,0,0,3000.0,NaN,...,0.0,-1.117043,NaN,-1.703222,-1.074260,0.333316,6.976312,NaN,5.634790,7.033457
4,20001,201705,201701,201912,5,36,0,0,3000.0,NaN,...,0.0,0.353285,NaN,2.865166,0.273781,-1.470536,7.315352,NaN,6.553933,7.347100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31357,21281,201704,201702,201708,3,7,1,1,NaN,NaN,...,1.0,-0.670410,NaN,NaN,NaN,1.850294,0.000000,NaN,NaN,NaN
31358,21281,201705,201702,201708,4,7,1,0,NaN,NaN,...,1.0,-0.670410,NaN,NaN,NaN,-0.707610,0.000000,NaN,NaN,NaN
31359,21281,201706,201702,201708,5,7,1,0,NaN,NaN,...,1.0,2.237248,NaN,1.414214,1.397345,0.272537,0.087406,NaN,2.197225,0.100198
31360,21281,201707,201702,201708,6,7,1,0,NaN,NaN,...,1.0,-0.670410,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN


In [4]:
%pip uninstall -y fbprophet
%pip install --quiet --upgrade prophet

import os
import sys

df = pd.DataFrame({
    'ds': pd.date_range(start='2020-01-01', periods=10),
    'y': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
})

model = Prophet(yearly_seasonality=True)
model.fit(df)  # ¡No debería dar errores!
print("Modelo entrenado correctamente")



Note: you may need to restart the kernel to use updated packages.


Note: you may need to restart the kernel to use updated packages.


AttributeError: 'Prophet' object has no attribute 'stan_backend'

In [10]:
def get_prophet_features(df, target_col):
    """
    Crea features de Prophet.
    target_col: Columna objetivo para el modelo Prophet.
    Esta función utiliza Prophet para extraer componentes aditivos y multiplicativos de la serie temporal
    y los combina con el DataFrame original.
    """
    
    ruta_archivo = f"./features_prophet_completo_{target_col}.csv"
    
    if os.path.exists(ruta_archivo) and ruta_archivo.endswith('.csv'):        
        features_final = pd.read_csv(ruta_archivo, sep=',', encoding='utf-8')
        features_final['ds'] = pd.to_datetime(features_final['ds']).dt.strftime('%Y%m').astype(int)
        features_final.drop(columns=['y','yhat1','yhat2'], inplace=True)
        df = df.merge(features_final, on=['periodo', 'product_id'], how='left')
        return df
    
    
    date_col = 'periodo'
    product_col = 'product_id'
    


    df['ds'] = pd.to_datetime(df[date_col].astype(str), format='%Y%m')
    df = df.sort_values([product_col, 'ds'])

    all_features = []

    for product in df[product_col].unique():
        product_df = df[df[product_col] == product].copy()
        
        # --- Modelo ADITIVO ---
        model_add = Prophet(
            yearly_seasonality=True,
            weekly_seasonality=False,
            daily_seasonality=False,
            seasonality_mode='additive'
        )
        model_add.fit(product_df[['ds', target_col]].rename(columns={target_col: 'y'}))
        forecast_add = model_add.make_future_dataframe(periods=0)
        components_add = model_add.predict(forecast_add)
        
        # --- Modelo MULTIPLICATIVO ---
        model_mult = Prophet(
            yearly_seasonality=True,
            weekly_seasonality=False,
            daily_seasonality=False,
            seasonality_mode='multiplicative'
        )
        model_mult.fit(product_df[['ds', target_col]].rename(columns={target_col: 'y'}))
        forecast_mult = model_mult.make_future_dataframe(periods=0)
        components_mult = model_mult.predict(forecast_mult)
        
        # --- Combinar componentes ---
        features = pd.DataFrame({'ds': components_add['ds']})
        
        # Extraer componentes ADITIVOS
        features['trend_add'] = components_add['trend']
        features['yearly_add'] = components_add['yearly']
        features['additive_terms'] = components_add['additive_terms']  # Suma de todos los términos aditivos
        
        # Extraer componentes MULTIPLICATIVOS (solo existen en mode multiplicativo)
        features['trend_mult'] = components_mult['trend']
        features['yearly_mult'] = components_mult['yearly']
        features['multiplicative_terms'] = components_mult['multiplicative_terms']  # Producto de términos multiplicativos
        
        features['product_id'] = product
        
        # Escalar features
        scaler = StandardScaler()
        feature_cols = ['trend_add', 'yearly_add', 'additive_terms', 
                        'trend_mult', 'yearly_mult', 'multiplicative_terms']
        features[feature_cols] = scaler.fit_transform(features[feature_cols])
        
        all_features.append(features)

    # Combinar todos los productos
    prophet_features = pd.concat(all_features)
    
    prophet_features.to_csv(ruta_archivo, index=False, sep=',', encoding='utf-8')

    # Unir con los datos originales
    df = df.merge(prophet_features, on=['ds', 'product_id'])
    
    return df

In [11]:
##### TARDA 66 MINUTOS EN EJECUTARSE
columns = ['tn_zscore', 'stock_final_zscore', 'cust_request_qty_zscore', 'cust_request_tn_zscore']

if ESTOY_EN_KAGGLE:
    df = get_prophet_features(df, 'tn_zscore') 
    df = get_prophet_features(df, 'stock_final_zscore') 
    df = get_prophet_features(df, 'cust_request_qty_zscore') 
    df = get_prophet_features(df, 'cust_request_tn_zscore') 

    # df = get_prophet_features_sin_data_leakage(df, 'tn_zscore', train_mask = (df['periodo'] <= 201909))

AttributeError: 'Prophet' object has no attribute 'stan_backend'